In [330]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

In [331]:
print('USE CUDA:',torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('CURRENT DEVICE:',device)

USE CUDA: False
CURRENT DEVICE: cpu


In [332]:
batch_size = 64
total_epoch = 100
learning_rate = 1e-4

In [333]:
df = pd.read_csv('min_max.csv')

training_df = df.iloc[:int(len(df)*0.8),:] 
val_df = df.iloc[int(len(df)*0.8):,:]

In [334]:
class CustomDataset(Dataset):
    def __init__(self,df):
        self.features = df.iloc[:,:-1].values
        self.features = torch.FloatTensor(self.features)
        self.labels = df.iloc[:,-1].values
        self.labels = torch.LongTensor(self.labels)

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        x = self.features[idx]
        y = self.labels[idx]
        return x,y

In [335]:
training_dataset = CustomDataset(training_df)
val_dataset = CustomDataset(val_df)

train_loader = DataLoader(training_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)
val_loader = DataLoader(val_dataset,batch_size=len(val_dataset))

total_batch = len(train_loader)

In [336]:
class DNN1(nn.Module):
    def __init__(self):
        super(DNN1,self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(52,128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.layer2 = nn.Sequential(
            nn.Linear(128,256),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.layer3 = nn.Sequential(
            nn.Linear(256,512),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.layer4 = nn.Sequential(
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.layer5 = nn.Sequential(
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.layer6 = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.layer7 = nn.Sequential(
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.layer8 = nn.Sequential(
            nn.Linear(32,3)
        )

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        return out

In [337]:
class DNN2(nn.Module):
    def __init__(self):
        super(DNN2,self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(52,512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.layer2 = nn.Sequential(
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.layer3 = nn.Sequential(
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.layer4 = nn.Sequential(
            nn.Linear(512,3)
        )


    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return out


In [338]:
total_epoch = 100
learning_rate = 1e-3


In [339]:
model = DNN2().to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

old_val_accuracy = -1

In [340]:
print('-'*40+'Start learning!'+'-'*40)
for epoch in tqdm(range(total_epoch)):
    epoch_avg_loss = 0.0
    epoch_accuracy_count = 0
    for num,data in enumerate(train_loader):
        optimizer.zero_grad()
        feature_vec, labels = data
        
        feature_vecs = feature_vec.to(device)
        labels = labels.to(device)
        
        output = model(feature_vecs)
        loss =  loss_function(output,labels)

        epoch_accuracy_count += (labels==output.argmax(dim=-1)).sum().item()  #미니배치에서의 정답 개수
        epoch_avg_loss += (loss/total_batch)

        loss.backward()
        optimizer.step()
    
    epoch_accuracy_rate = epoch_accuracy_count/(len(train_loader)*batch_size)*100

    if epoch%10 == 0:
        
        
        model.eval()
        with torch.no_grad():
            for val_num,val_data in enumerate(val_loader):
                val_features ,val_labels = val_data
                val_features = val_features.to(device)
                val_labels = val_labels.to(device)
                
                val_output = model(val_features)
                val_accracy_count = (val_output.argmax(dim=-1)==val_labels).sum()

                val_accracy_rate = val_accracy_count/len(val_dataset)*100
        model.train()
        
        print(f'[EPOCH: {str(epoch).rjust(3)}/{total_epoch}] [LOSS: {epoch_avg_loss:5f}] [ACCURACY: {epoch_accuracy_rate:2f}%] [VAL_ACCURACY: {val_accracy_rate:2f}]')    
print('-'*40+'learning finished!'+'-'*40)

----------------------------------------Start learning!----------------------------------------


  0%|          | 0/100 [00:00<?, ?it/s]

[EPOCH:   0/100] [LOSS: 0.884307] [ACCURACY: 64.308712%] [VAL_ACCURACY: 63.265305]
[EPOCH:  10/100] [LOSS: 0.803309] [ACCURACY: 68.740530%] [VAL_ACCURACY: 67.517006]
[EPOCH:  20/100] [LOSS: 0.757792] [ACCURACY: 70.440341%] [VAL_ACCURACY: 68.102798]
[EPOCH:  30/100] [LOSS: 0.730007] [ACCURACY: 71.212121%] [VAL_ACCURACY: 68.216171]
[EPOCH:  40/100] [LOSS: 0.715761] [ACCURACY: 71.543561%] [VAL_ACCURACY: 69.066513]
[EPOCH:  50/100] [LOSS: 0.695313] [ACCURACY: 72.225379%] [VAL_ACCURACY: 69.009827]
[EPOCH:  60/100] [LOSS: 0.681989] [ACCURACY: 72.646780%] [VAL_ACCURACY: 68.707481]
[EPOCH:  70/100] [LOSS: 0.670357] [ACCURACY: 73.001894%] [VAL_ACCURACY: 69.255478]
[EPOCH:  80/100] [LOSS: 0.660249] [ACCURACY: 73.139205%] [VAL_ACCURACY: 68.972031]
[EPOCH:  90/100] [LOSS: 0.657412] [ACCURACY: 73.683712%] [VAL_ACCURACY: 68.934235]
----------------------------------------learning finished!----------------------------------------


In [341]:
print(val_output.argmax(dim=-1))

tensor([2, 2, 2,  ..., 2, 2, 2])
